In [1]:
import pandas as pd
from Bio import SeqIO
from Bio.SeqFeature import FeatureLocation
from Bio.Seq import UndefinedSequenceError
import argparse
import os
import subprocess
import numpy as np
import random
random.seed(42)

TODO:
- Compare the isolation hosts that I used to obtain to the ones that I obtain now
- For that I may need to run the old engineer features again 

READ:

Bueno, pues estaba utilizando el archivo que no era para sacar los ids y hosts. Estaba usando un archivo que se llama como vcontact2 y creo que eso tampoco es correcto, ademas de que me dejaba como 5000 fagos sin mapear

Ahora he usado el archivo que se llama itol annotations y en este caso solo me dejo 99 fagos sin mapear. El archivo se llama final_hopefully_engineer_features. El codigo esta al final de 01_new_engineering

In [33]:
import pickle
import pandas as pd
# df = pd.read_csv('../data/test/test_engineer_features.csv')
# df = pd.read_csv('../data/test/nan_engineer_features.csv')
# df = pd.read_csv('../data/test/fixing_engineer_features.csv')

# old = pd.read_csv('../data/test/old_engineer_features.csv')
df = pd.read_csv('../data/test/final_hopefully_engineer_features.csv') # missing 99
df2 = pd.read_csv('../data/test/fixing_engineer_features.csv') # missing 5k
print("Shape before filtering", df.shape)
# print(df.columns)
# df.head()

Shape before filtering (26286, 28)


Some preliminary exploration of the data. There are 99 entries for which we have no ids or host_inphared

In [21]:
df[df['host_inphared'].isna()]
# df[df['id'] == "PZACG"]

# df[df["length"] == 19366.0]


,id,host_inphared,length,jumbophage,gc_perc,CDS,frame_positive,frame_negative,cds_coding_density,positive_strand_%,...,lysis,nucleotide_metabolism,other,tRNAs,tail,tmRNAs,transcription,unkown_function,transl_table,sequence
0,NaN,NaN,53332.0,0,64.53,90.0,36.0,54.0,90.11,40.00,...,3.0,11.0,3.0,1.0,13.0,0.0,3.0,44.0,11.0,NaN
1,NaN,NaN,32618.0,0,39.22,42.0,7.0,35.0,89.29,16.67,...,1.0,2.0,1.0,0.0,10.0,0.0,1.0,17.0,11.0,NaN
2,NaN,NaN,94878.0,0,28.51,89.0,50.0,39.0,94.70,56.18,...,1.0,8.0,2.0,3.0,1.0,0.0,0.0,74.0,11.0,NaN
3,NaN,NaN,90023.0,0,29.18,87.0,41.0,46.0,92.83,47.13,...,0.0,11.0,0.0,2.0,0.0,0.0,0.0,72.0,11.0,NaN
4,NaN,NaN,97065.0,0,29.27,92.0,47.0,45.0,94.52,51.09,...,1.0,10.0,2.0,0.0,2.0,0.0,0.0,74.0,11.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17657,NaN,NaN,14504.0,0,40.31,25.0,4.0,21.0,61.61,16.00,...,0.0,2.0,1.0,23.0,0.0,0.0,0.0,22.0,11.0,NaN
20213,NaN,NaN,17529.0,0,34.81,14.0,0.0,14.0,92.10,0.00,...,2.0,0.0,1.0,0.0,4.0,0.0,0.0,6.0,11.0,NaN
20215,NaN,NaN,10647.0,0,33.50,15.0,0.0,15.0,92.73,0.00,...,0.0,4.0,1.0,0.0,0.0,0.0,3.0,4.0,11.0,NaN
20216,NaN,NaN,16005.0,0,34.38,13.0,2.0,11.0,96.92,15.38,...,2.0,0.0,1.0,0.0,4.0,0.0,0.0,6.0,11.0,NaN


We have these entries in the dataset that was mapped to the vcontact2

In [29]:
# df2[df2['host_inphared'].isna()]
# df2.iloc[17657]

In [5]:
# old = pd.read_csv('../data/test/old_engineer_features.csv')


In [24]:
# old[old["length"] == 19366.0]

In [34]:
# Old - when we had isolation host
# discarded = df[(df['host_inphared'] == 'Unspecified') | (df['isolation_host_inphared'] == 'Unspecified')]
# df = df[df['host_inphared'] != "Unspecified"]
# df = df[df['isolation_host_inphared'] != "Unspecified"]
# df['isolation_host_inphared'] = df['isolation_host_inphared'].str.split().str[:2].str.join(' ')
# df['host_inphared'] = df['host_inphared'].replace('Enteroccous', 'Enterococcus')
# df.loc[df['isolation_host_inphared'] == 'Salmonella enterica', 'host_inphared'] = 'Salmonella'
# df.loc[df['isolation_host_inphared'] == 'Salmonella typhimurium', 'host_inphared'] = 'Salmonella'
# df.loc[df['isolation_host_inphared'] == 'Escherichia coli', 'host_inphared'] = 'Escherichia'
# df.loc[df['isolation_host_inphared'] == 'E. coli', 'host_inphared'] = 'Escherichia'
# df.loc[df['isolation_host_inphared'] == 'Escherichia coli,', 'host_inphared'] = 'Escherichia'
# df.loc[df['isolation_host_inphared'] == 'Shigella flexneri', 'host_inphared'] = 'Shigella'


# print(f"After removing unspecified host_inphared and isolation_host_inphared, the shape of the discarded is {discarded.shape}")
# print(f"Shape after filtering unspecified hosts for 'df': {df.shape}")


# New - without isolation host
discarded = df.copy(deep="True")
discarded = discarded[(discarded['host_inphared'] == 'Unspecified')]
df = df[df['host_inphared'] != "Unspecified"]
df['host_inphared'] = df['host_inphared'].replace('Enteroccous', 'Enterococcus')


print(f"After removing unspecified host_inphared, the shape of the discarded is {discarded.shape}")
print(f"Shape after filtering unspecified hosts for 'df': {df.shape}")

After removing unspecified host_inphared, the shape of the discarded is (2888, 28)
Shape after filtering unspecified hosts for 'df': (23398, 28)


# Gram assignation

In the version of the code where I used the inphared top hit files, I had a columnc alled isolation_host_inphared that gave me specific species names for each phage. That is no longer the case, and now I can only really rely on the "host_inphared" column, which is usually the host. My all_gram dataset has staining information down to the species level, but that is no longer useful. I need to collapse the df and assign single staining to the genus level. This is a problem because some genera have different staining in different species. What I am going to do is for each Genus obtain the staining that corresponds to more than 75% of its species. If the value is lower than that (for example, we only have 2 entries and one is positive but the other is negative), we exclude the genus and we will not take it into account

In [35]:
all_gram = pd.read_csv("../data/interim/gram_staining/exploration/all_gram_stains_fixed_species.csv")
all_gram[all_gram["Genus"] == "Desulfotomaculum"]["Genus"].value_counts()
all_gram[all_gram["Genus"] == "Desulfotomaculum"]

# Calculate the percentage of positive and negative "gram stain" assignations for each genus
genus_percentages = all_gram.groupby('Genus')['Gram stain'].value_counts(normalize=True).unstack()

# Filter genera where either the percentage of positive or negative assignations is above 0.75
genus_above_75_percent = genus_percentages[((genus_percentages['negative'] >= 0.75) | (genus_percentages['positive'] >= 0.75))]

# Create a new DataFrame with just the "Genus" and "Gram stain" columns
gram_75 = pd.DataFrame({
    'Genus': genus_above_75_percent.index,
    'Gram stain': genus_above_75_percent.idxmax(axis=1)  # Get the index of the maximum value along the columns
}).reset_index(drop=True)

new_entry = pd.DataFrame({'Genus': ['Enterobacteria'], 'Gram stain': ['negative']})
gram_75 = pd.concat([gram_75, new_entry], ignore_index=True)

print("New DataFrame with Genus and majority Gram stain:")
# gram_75.to_csv("../data/interim/gram_staining/exploration/gram_75_genus.csv")
gram_75[gram_75["Genus"] == "Enterobacteria"]

New DataFrame with Genus and majority Gram stain:


,Genus,Gram stain
2119,Enterobacteria,negative


In [36]:
# Apply gram staining to the "host_inphared" column using the "gram_75" DataFrame
df['staining'] = df['host_inphared'].map(gram_75.set_index('Genus')['Gram stain'])

print("Excluded rows where 'host_inphared' entry is not in 'gram_75':")

# Find rows where the "host_inphared" entry is not in "gram_75"
excluded_rows_df = df[df['staining'].isna()]

# Append the excluded rows to the discarded DataFrame for df
discarded = pd.concat([discarded, excluded_rows_df], ignore_index=True)
# del excluded_rows_df

print(f"Shape of 'df' after mapping Genus to inphared_host considering the 75% rule': {df.shape}")
# df[df["staining"].isna()]["host_inphared"].value_counts() # with this I can see which genera are still to be assigned a stain 


# Calculate the percentage of non-NaN rows in the "staining" column
non_nan_percentage = (df['staining'].notna().sum() / len(df['staining'])) * 100

print("Percentage of phages assigned a class based solely on Genus with the all_gram dictionary and 75% rule for df: ", non_nan_percentage)

Excluded rows where 'host_inphared' entry is not in 'gram_75':
Shape of 'df' after mapping Genus to inphared_host considering the 75% rule': (23398, 29)
Percentage of phages assigned a class based solely on Genus with the all_gram dictionary and 75% rule for df:  88.50756474912386


To avoid inconsistencies when using Thomas' dictionary, I only apply it to the rows where staining is.na

In [37]:
with open('../data/interim/gram_staining/updated_gram_pos_neg_dict.pkl', 'rb') as file:
    phage_dict = pickle.load(file)

file.close()

# Create a new genus_to_stain with updated values
phage_dict = {key: 'negative' if value == 'neg' else 'positive' for key, value in phage_dict.items()}
phage_dict = pd.DataFrame.from_dict(phage_dict, orient='index', columns=['Gram stain'])

# Set the name of the index to 'genus'
phage_dict.index.name = 'Genus'

phage_dict = phage_dict.reset_index()

# Apply gram staining to the rows with NaN values in the "staining" column using the "phage_dict" DataFrame
df.loc[df['staining'].isna(), 'staining'] = df.loc[df['staining'].isna(), 'host_inphared'].map(phage_dict.set_index('Genus')['Gram stain'])

print("Excluded rows where 'host_inphared' entry is not in 'phage_dict':")

# Find rows where the "host_inphared" entry is not in "phage_dict"
excluded_rows_df = df[df['staining'].isna()]

# Append the excluded rows to the discarded DataFrame for df
discarded = pd.concat([discarded, excluded_rows_df], ignore_index=True)
# del excluded_rows_df

print(f"Shape of 'df' after mapping Genus to inphared_host considering the 75% rule': {df.shape}")
# df[df["staining"].isna()]["host_inphared"].value_counts() # with this I can see which genera are still to be assigned a stain 


# Calculate the percentage of non-NaN rows in the "staining" column
non_nan_percentage = (df['staining'].notna().sum() / len(df['staining'])) * 100

print("Percentage of phages assigned a class adding the phage dict: ", non_nan_percentage)

Excluded rows where 'host_inphared' entry is not in 'phage_dict':
Shape of 'df' after mapping Genus to inphared_host considering the 75% rule': (23398, 29)
Percentage of phages assigned a class adding the phage dict:  98.04256774083255


After the staining, I am still losing 99 phages, which are the phages for which I did not have a mapping in the beginning

In [38]:
# Displaying rows where the 'staining' column is not NaN for df
print("Rows where the 'staining' column is not NaN for df:")
df[~df['staining'].isna()]

print("Rows where the 'staining' column is  NaN for df:")
df[df['host_inphared'].isna()]


Rows where the 'staining' column is not NaN for df:
Rows where the 'staining' column is  NaN for df:


,id,host_inphared,length,jumbophage,gc_perc,CDS,frame_positive,frame_negative,cds_coding_density,positive_strand_%,...,nucleotide_metabolism,other,tRNAs,tail,tmRNAs,transcription,unkown_function,transl_table,sequence,staining
0,NaN,NaN,53332.0,0,64.53,90.0,36.0,54.0,90.11,40.00,...,11.0,3.0,1.0,13.0,0.0,3.0,44.0,11.0,NaN,NaN
1,NaN,NaN,32618.0,0,39.22,42.0,7.0,35.0,89.29,16.67,...,2.0,1.0,0.0,10.0,0.0,1.0,17.0,11.0,NaN,NaN
2,NaN,NaN,94878.0,0,28.51,89.0,50.0,39.0,94.70,56.18,...,8.0,2.0,3.0,1.0,0.0,0.0,74.0,11.0,NaN,NaN
3,NaN,NaN,90023.0,0,29.18,87.0,41.0,46.0,92.83,47.13,...,11.0,0.0,2.0,0.0,0.0,0.0,72.0,11.0,NaN,NaN
4,NaN,NaN,97065.0,0,29.27,92.0,47.0,45.0,94.52,51.09,...,10.0,2.0,0.0,2.0,0.0,0.0,74.0,11.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17657,NaN,NaN,14504.0,0,40.31,25.0,4.0,21.0,61.61,16.00,...,2.0,1.0,23.0,0.0,0.0,0.0,22.0,11.0,NaN,NaN
20213,NaN,NaN,17529.0,0,34.81,14.0,0.0,14.0,92.10,0.00,...,0.0,1.0,0.0,4.0,0.0,0.0,6.0,11.0,NaN,NaN
20215,NaN,NaN,10647.0,0,33.50,15.0,0.0,15.0,92.73,0.00,...,4.0,1.0,0.0,0.0,0.0,3.0,4.0,11.0,NaN,NaN
20216,NaN,NaN,16005.0,0,34.38,13.0,2.0,11.0,96.92,15.38,...,0.0,1.0,0.0,4.0,0.0,0.0,6.0,11.0,NaN,NaN


I want to remove the NaNs

In [42]:
df = df[~df['staining'].isna()]
df.columns

Index(['id', 'host_inphared', 'length', 'jumbophage', 'gc_perc', 'CDS',
       'frame_positive', 'frame_negative', 'cds_coding_density',
       'positive_strand_%', 'negative_strand_%', 'CARD_AMR_Genes', 'CRISPRs',
       'VFDB_Virulence_Factors', 'connector', 'head_packaging',
       'host_takeover', 'integration and excision', 'lysis',
       'nucleotide_metabolism', 'other', 'tRNAs', 'tail', 'tmRNAs',
       'transcription', 'unkown_function', 'transl_table', 'sequence',
       'staining'],
      dtype='object')

Seeing if there was a problem in the parsing of the fiels

In [43]:
# Code to save the features with the Gram Class assignation
# Removing Host and Isolation Host
df = df[['id', 'staining', 'length',
       'jumbophage', 'gc_perc', 'CDS', 'frame_positive', 'frame_negative',
       'cds_coding_density', 'positive_strand_%', 'negative_strand_%',
       'CARD_AMR_Genes', 'CRISPRs', 'VFDB_Virulence_Factors', 'connector',
       'head_packaging', 'host_takeover', 'integration and excision', 'lysis',
       'nucleotide_metabolism', 'other', 'tRNAs', 'tail', 'tmRNAs',
       'transcription', 'unkown_function', 'transl_table', 'sequence']]

stain_output = "../data/test/fixed_features_with_gram_class.csv"
df.to_csv(stain_output, index=False)
print(f"---Gram staining assignation has been completed, and the data is stored in {stain_output}")

---Gram staining assignation has been completed, and the data is stored in ../data/test/fixed_features_with_gram_class.csv


: 